In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
df = pd.read_csv("data_customer.csv")
#data cleaning duplicate removal stage
#TransactionID is 100% unique the rest are repeated
#usually it will remove duplicates only if transactionID is repeated it will have no effect here
#no need for duplicte search since it is unique as a dataset
dupe_timestamps = df[df["Timestamp"].duplicated(keep=False)].sort_values("Timestamp")
print(f"Found {dupe_timestamps.shape[0]} duplicate timestamps.")
dupe_timestamps
df.to_csv("data_customer_noduplicates.csv", index=False)
print("Clean version saved: data_customer_noduplicates.csv")



Found 0 duplicate timestamps.
Clean version saved: data_customer_noduplicates.csv


In [42]:
#no duplicates found we can go to next step in cleaning
#ensuring every row have full valid info
import pandas as pd

df = pd.read_csv("data_customer_noduplicates.csv")
print("Dataset shape after removing duplicates:", df.shape)
df.head()
#dropping empty rows
before = df.shape[0]
df.dropna(how="all", inplace=True)
after = df.shape[0]
print(f"Removed {before - after} completely empty rows.")
#check for missing values
key_columns = ["CustomerID", "Timestamp", "Service"]
for col in key_columns:
    if col in df.columns:
        missing = df[col].isna().sum()
        print(f"{col}: {missing} missing values")
# Display any rows missing critical info
df[df[key_columns].isna().any(axis=1)]
#Remove rows with missing info like customerid service or timestamp
df = df.dropna(subset=["CustomerID", "Timestamp", "Service"])
print("Remaining rows:", df.shape)
#remove nonsense values rows 
mask = (
    (df["CustomerID"].astype(str).str.lower().isin(["nan", "none", "null", "na"])) |
    (df["Service"].astype(str).str.lower().isin(["unknown", "na", "none"]))
)
df = df[~mask]

print("Shape after removing placeholders:", df.shape)
#save the new cleaned set
df.to_csv("data_customer_cleaned.csv", index=False)
print(" Final cleaned dataset saved as data_customer_cleaned.csv")
#final verification
print("Remaining missing values:")
print(df.isna().sum())
print("\nFinal shape:", df.shape)
#our dataset now is fully cleaned 


Dataset shape after removing duplicates: (101, 12)
Removed 0 completely empty rows.
CustomerID: 1 missing values
Timestamp: 1 missing values
Service: 1 missing values
Remaining rows: (100, 12)
Shape after removing placeholders: (100, 12)
 Final cleaned dataset saved as data_customer_cleaned.csv
Remaining missing values:
TransactionID         0
Timestamp             0
DayOfWeek             0
BranchLocation        0
CustomerID            0
Gender                0
Service               0
Price (IDR)           0
BarberName            0
BarberTier            0
Rating (1-5)          0
Duration (Minutes)    0
dtype: int64

Final shape: (100, 12)


In [46]:
#some data standarization into english since the dataset is indonsian transform price into usd and serives into english
IDR_to_USD = 15500  # taken from internet sources
# Create a new column for USD
df["Price (USD)"] = df["Price (IDR)"] / IDR_to_USD
# Round for readability
df["Price (USD)"] = df["Price (USD)"].round(2)
print("Currency converted: IDR → USD")
df[["Price (IDR)", "Price (USD)"]].head(10)
#service translation from indo to english 
service_translation = {
    "Cukur Rambut": "Haircut",
    "Potong Jenggot": "Beard Trim",
    "Cukur & Cuci": "Haircut + Wash",
    "Cukur & Pijat": "Haircut + Massage",
    "Cukur Anak": "Kids Haircut",
    "Pewarnaan Rambut": "Hair Coloring",
    "Perawatan Rambut": "Hair Treatment"
}
# Apply translation only if values exist in dictionary
df["Service (English)"] = df["Service"].map(service_translation).fillna(df["Service"])
print("Service names translated to English (where possible).")
df[["Service", "Service (English)"]].drop_duplicates().head(10)
#translate genders into english
gender_map = {
    "Pria": "Male",
    "Wanita": "Female"
}
df["Gender (English)"] = df["Gender"].map(gender_map).fillna(df["Gender"])
df["Gender (English)"].value_counts()
#translate days of the week into english
day_map = {
    "Senin": "Monday",
    "Selasa": "Tuesday",
    "Rabu": "Wednesday",
    "Kamis": "Thursday",
    "Jumat": "Friday",
    "Sabtu": "Saturday",
    "Minggu": "Sunday"
}
df["DayOfWeek (English)"] = df["DayOfWeek"].map(day_map).fillna(df["DayOfWeek"])
df["DayOfWeek (English)"].value_counts()

#save updated version                                        
df.to_csv("data_customer_standardized.csv", index=False)
print("Saved standardized dataset as data_customer_standardized.csv")



Currency converted: IDR → USD
Service names translated to English (where possible).
Saved standardized dataset as data_customer_standardized.csv


In [47]:
#final dataset version fully translated and dropping non english col
import pandas as pd

df = pd.read_csv("data_customer_standardized.csv")
print("Shape before final cleanup:", df.shape)
df.head(3)
columns_to_drop = [
    "Price (IDR)",
    "Service",
    "Gender",
    "DayOfWeek"
]

# Drop only the ones that exist (safe drop)
df.drop(columns=[c for c in columns_to_drop if c in df.columns], inplace=True)
print("Shape after dropping old columns:", df.shape)
print("Remaining columns:")
print(df.columns.tolist())
#rename for simplicity
df.rename(columns={
    "Service (English)": "Service",
    "Gender (English)": "Gender",
    "DayOfWeek (English)": "DayOfWeek",
    "Price (USD)": "PriceUSD"
}, inplace=True)
df.head(5)
#final small check 
print("Remaining null values per column:")
print(df.isna().sum())
print("\nDuplicate rows:", df.duplicated().sum())
#saving final version
df.to_csv("data_customer_final_cleaned.csv", index=False)
print("dataset saved as data_customer_final_cleaned.csv")


Shape before final cleanup: (100, 16)
Shape after dropping old columns: (100, 12)
Remaining columns:
['TransactionID', 'Timestamp', 'BranchLocation', 'CustomerID', 'BarberName', 'BarberTier', 'Rating (1-5)', 'Duration (Minutes)', 'Price (USD)', 'Service (English)', 'Gender (English)', 'DayOfWeek (English)']
Remaining null values per column:
TransactionID         0
Timestamp             0
BranchLocation        0
CustomerID            0
BarberName            0
BarberTier            0
Rating (1-5)          0
Duration (Minutes)    0
PriceUSD              0
Service               0
Gender                0
DayOfWeek             0
dtype: int64

Duplicate rows: 0
dataset saved as data_customer_final_cleaned.csv
